In [10]:
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.measurements import label
from collections import deque
from advanced_line import *
from lesson_functions import *

# Loading Lane Finding Calibration and Car Detection Model

In [12]:
# loading camera calibration
pickle_fname = 'output_images/camera_cal_mtx_dist.p'
mtx, dist = load_camera_cal_params(pickle_fname)

## Load model params
#svc_rbf-params.pk - svc-rbf

with open('svc_params_YCrCb_hogALL_orient9_size32_pix8_cell2.pk', 'rb') as pfile:
    pickle_data = pickle.load(pfile)
    for key in pickle_data:
        exec(key + "= pickle_data['" + str(key) + "']")
    del pickle_data


In [13]:
print( color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size, hist_bins)

YCrCb 9 8 2 ALL (32, 32) 32


# Video Processing

In [14]:
def process_frame_video(img):
    # Undistort
    img_undist = pipeline_undistort(img, mtx, dist)
    
    # Perspective Transform
    binary = pipeline_transform(img_undist)
   
    # bird view
    warped, M = birds_eye_view(binary)
    
    
    # fit lines
    r_meters=0
    right_from_center_m = 0
    img_lines =[] 
    l_fitx =[] 
    r_fitx=[] 
    ploty = []
    l_fit =[] 
    r_fit=[] 

    if len(l_line.current_fit) == 0 and len(r_line.current_fit)==0:
        img_lines, r_meters, right_from_center_m, l_fitx, r_fitx, ploty, l_fit, r_fit = fit_lines(warped)
    else :
        img_lines, r_meters, right_from_center_m, l_fitx, r_fitx, ploty, l_fit, r_fit = \
            fit_lines(warped,l_line.best_fit, r_line.best_fit)
    
    # invalidate both fits if the difference in their x-intercepts isn't around 350 px (+/- 100 px)
    
    if l_fit is not None and r_fit is not None:
        # calculate x-intercept (bottom of image, x=image_height) for fits
        h = img.shape[0]
        l_fit_x_int = l_fit[0]*h**2 + l_fit[1]*h + l_fit[2]
        r_fit_x_int = r_fit[0]*h**2 + r_fit[1]*h + r_fit[2]
        x_int_diff = abs(r_fit_x_int-l_fit_x_int)
        if abs(350 - x_int_diff) > 100:
            l_fit = None
            r_fit = None
            
    l_line.add_fit(l_fit)
    r_line.add_fit(r_fit)
    Minv = np.linalg.inv(M)
    #print(len(l_line.best_fit),len(r_line.best_fit))   
    
    #r_fit=r_line.best_fit
    #l_fit=l_line.best_fit
    
    if l_line.best_fit is not None and r_line.best_fit is not None : 
        l_fitx = l_line.best_fit[0]*ploty**2 + l_line.best_fit[1]*ploty + l_line.best_fit[2]
        r_fitx = r_line.best_fit[0]*ploty**2 + r_line.best_fit[1]*ploty + r_line.best_fit[2]
        r_meters, right_from_center_m=calculate_radius(warped, ploty, l_line.best_fit, r_line.best_fit, l_fitx, r_fitx)
        
    # draw the current best fit if it exists
    #if l_fit is not None and r_fit is not None:
    #final_output = to_real_world_space(img_undist, img_lines, Minv, l_fitx, r_fitx, ploty)
    final_output = to_real_world_space(img, img_lines, Minv, l_fitx, r_fitx, ploty)
    
    # Annotate image with text
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = "Curve Radius: {:.0f} m".format(r_meters)
    cv2.putText(final_output, text, (50,50), font, 1, (255,255,255), 2)
    text = "Offset from center: {:.2f} m".format(right_from_center_m)
    cv2.putText(final_output, text, (50,100), font, 1, (255,255,255), 2)
    
    pts_l = np.array([np.transpose(np.vstack([l_fitx, ploty]))])
    cv2.polylines(img_lines, np.int32([pts_l]), isClosed=False, color=(255,255,0), thickness=8)

    pts_r = np.array([np.transpose(np.vstack([r_fitx, ploty]))])
    cv2.polylines(img_lines, np.int32([pts_r]), isClosed=False, color=(255,255,0), thickness=8)
    
    rectangles = []

    const_list = [(380, 480, 1), (400, 600, 1.5), (500, 700, 2.5)]
    
    for (ystart, ystop, scale) in const_list:
        rectangles.append(find_cars(img, ystart, ystop, scale, svc, X_scaler, color_space, orient, pix_per_cell, 
                                    cell_per_block, hog_channel, spatial_size, hist_bins,vis=False))

    rectangles = [item for sublist in rectangles for item in sublist] 
    
    # add detections to the history
    if len(rectangles) > 0:
        det.add_rects(rectangles)
        heat = np.zeros_like(img[:,:,0]).astype(np.float)
        heat = add_heat(heat,rectangles)
        #heat = apply_threshold(heat,1)
        heat[heat>0]=1
        det.add_heat(heat)
    
    heatmap_cut = np.zeros_like(img[:,:,0])
    for heat in det.prev_heat:
        heatmap_cut = heatmap_cut+heat
    
    # LUV 32
    if color_space=='LUV':
        heatmap_cut = apply_threshold(heatmap_cut, 27)
    elif color_space=='YCrCb':
        heatmap_cut = apply_threshold(heatmap_cut, 15)
        
    heatmap_cut[heatmap_cut>0]=1
    
    heatmap_img = np.zeros_like(img[:,:,0])
    i = 0
    for rect_set in det.prev_rects:
        rect_set_new=check_heat(heatmap_cut,rect_set)
        if len(rect_set_new) > 0:
            i += 1
            heatmap_img = add_heat(heatmap_img, rect_set_new)
            
    if color_space=='LUV':
        heatmap_img = apply_threshold(heatmap_img, i)
    elif color_space=='YCrCb':
        heatmap_img = apply_threshold(heatmap_img,i//2)
        
    # labelling
     
    labels = label(heatmap_img)
    draw_img = draw_labeled_bboxes(final_output, labels)

    #return np.hstack((final_output, img_lines))
    return final_output


In [20]:
# Define a class to store data from video
class Vehicle_Detect():
    def __init__(self):
        # history of rectangles previous n frames
        self.prev_rects = []
        self.prev_heat = []
        
    def add_rects(self, rects):
        self.prev_rects.append(rects)
        if len(self.prev_rects) > 30:
            # throw out oldest rectangle set(s)
            self.prev_rects = self.prev_rects[len(self.prev_rects)-30:]
    
    def add_heat(self, heat):
        self.prev_heat.append(heat)
        if len(self.prev_heat) > 30:
            self.prev_heat = self.prev_heat[len(self.prev_heat)-30:]
        

In [16]:
l_line = Line()
r_line = Line()
det = Vehicle_Detect()

test_out_file = 'test_video_out_'+str(color_space)+'32_withlane.mp4'
clip_test = VideoFileClip('test_video.mp4')
clip_test_out = clip_test.fl_image(process_frame_video)
%time clip_test_out.write_videofile(test_out_file, audio=False)

[MoviePy] >>>> Building video test_video_out_YCrCb32_withlane.mp4
[MoviePy] Writing video test_video_out_YCrCb32_withlane.mp4


 97%|█████████▋| 38/39 [00:38<00:00,  1.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_out_YCrCb32_withlane.mp4 

CPU times: user 43.5 s, sys: 10 s, total: 53.5 s
Wall time: 39.9 s


In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_out_file))

In [18]:
l_line = Line()
r_line = Line()
det = Vehicle_Detect()

project_out_file = 'project_video_out_'+str(color_space)+'32_withlane.mp4'
clip_project = VideoFileClip('project_video.mp4')
clip_project_out = clip_project.fl_image(process_frame_video)
%time clip_project_out.write_videofile(project_out_file, audio=False)

[MoviePy] >>>> Building video project_video_out_YCrCb32_withlane.mp4
[MoviePy] Writing video project_video_out_YCrCb32_withlane.mp4


100%|█████████▉| 1260/1261 [20:58<00:00,  1.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out_YCrCb32_withlane.mp4 

CPU times: user 24min 1s, sys: 5min 44s, total: 29min 45s
Wall time: 20min 59s


In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_out_file))